In [ ]:
!pip install surprise


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate



In [ ]:
movies= pd.read_csv('movies_metadata.csv')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46628 entries, 0 to 46627
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  46628 non-null  object 
 1   belongs_to_collection  4574 non-null   object 
 2   budget                 46628 non-null  object 
 3   genres                 46628 non-null  object 
 4   homepage               8009 non-null   object 
 5   id                     46628 non-null  int64  
 6   imdb_id                46611 non-null  object 
 7   original_language      46617 non-null  object 
 8   original_title         46628 non-null  object 
 9   overview               46628 non-null  object 
 10  popularity             46624 non-null  object 
 11  poster_path            46229 non-null  object 
 12  production_companies   46624 non-null  object 
 13  production_countries   46624 non-null  object 
 14  release_date           46540 non-null  object 
 15  re

In [ ]:
movies.iloc[0:3].transpose()

,0,1,2
adult,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect..."
budget,30000000,65000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
homepage,http://toystory.disney.com/toy-story,NaN,NaN
id,862,8844,15602
imdb_id,tt0114709,tt0113497,tt0113228
original_language,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...


In [ ]:
movies.shape

(45466, 24)

In [ ]:
credits= pd.read_csv('credits.csv')



In [ ]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
credits.shape

(45476, 3)

In [ ]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [ ]:
credits.iloc[0:3]

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602


In [ ]:
keywords= pd.read_csv('keywords.csv')
links= pd.read_csv('links_small.csv')
ratings= pd.read_csv('ratings_small.csv')


In [ ]:
keywords.head()


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
keywords.shape

(46419, 2)

In [ ]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
links.shape

(9125, 3)

In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
ratings.shape

(100004, 4)

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
# Content-based recommender

movies['overview'] = movies['overview'].fillna('')
movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [ ]:

pd.DataFrame({'feature':movies.dtypes.index, 'dtype':movies.dtypes.values})

,feature,dtype
0,adult,object
1,belongs_to_collection,object
2,budget,object
3,genres,object
4,homepage,object
5,id,int64
6,imdb_id,object
7,original_language,object
8,original_title,object
9,overview,object


In [ ]:
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan


In [ ]:
movies['id'] = movies['id'].apply(convert_int)
movies[movies['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
movies = movies.drop([19730, 29503, 35587])


In [ ]:
#Using movie description, taglines, keywords, cast, director and genres
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies['id'] = movies['id'].astype('int')



In [ ]:
movies.shape

(45463, 24)

In [ ]:
movies = movies.merge(credits, on='id')
movies = movies.merge(keywords, on='id')

In [ ]:
s_movies = movies[movies['id'].isin(links)]
s_movies.shape

(9219, 27)

In [ ]:
s_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
s_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [ ]:
s_movies['crew'].head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [ ]:
s_movies['tagline'] = s_movies['tagline'].fillna('')
s_movies['description'] = s_movies['overview'] + s_movies['tagline']
s_movies['description'] = s_movies['description'].fillna('')


In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(s_movies['description'])


In [ ]:
tfidf_matrix.shape


(9219, 268124)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [ ]:
cosine_sim[0]

array([1.        , 0.00681816, 0.        , ..., 0.        , 0.        ,
       0.00479058])

In [ ]:
s_movies = s_movies .reset_index()
titles =s_movies ['title']
indices = pd.Series(s_movies .index, index=s_movies ['title'])
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


In [ ]:
get_recommendations('Toy Story').head(10)

2522               Toy Story 2
7629               Toy Story 3
6267    The 40 Year Old Virgin
2567           Man on the Moon
6708              Factory Girl
4757    What's Up, Tiger Lily?
910      Rebel Without a Cause
6635    For Your Consideration
5043          Rivers and Tides
1619                 Condorman
Name: title, dtype: object

In [ ]:
get_recommendations('Asoka').head(10)

6844        I Could Never Be Your Woman
8760                The Look of Silence
3060    Sinbad and the Eye of the Tiger
3223               The Living Daylights
232                           Star Wars
1163                             Hamlet
4928             The Matrix Revolutions
3519                 The Lion in Winter
8881                     American Heist
6985                Under the Same Moon
Name: title, dtype: object

In [ ]:
get_recommendations('Monsoon Wedding').head(10)

9098                    Hyena Road
1689                  Grand Canyon
4584                Salaam Bombay!
56      The Indian in the Cupboard
6357                          Rent
5980                         11:14
2806              Madame Sousatzka
4740                 Swimming Pool
4804                          Fire
9184                  Mother's Day
Name: title, dtype: object

In [ ]:
get_recommendations('Jurassic Park').head(10)

8819                          Jurassic World
404                               Free Willy
8492                              Austenland
1837            History of the World: Part I
2872             Carnosaur 3: Primal Species
878                                    Johns
2256             National Lampoon's Vacation
1388    3 Ninjas: High Noon at Mega Mountain
3589                    The Land Before Time
9018                                Vacation
Name: title, dtype: object

In [ ]:
get_recommendations('Carnosaur 3: Primal Species').head(10)

2870                     Carnosaur
2871                   Carnosaur 2
427                  Jurassic Park
3589          The Land Before Time
144     Die Hard: With a Vengeance
4220                        S1m0ne
6578                     Idiocracy
4662              2 Fast 2 Furious
7986                     The Thing
9009             The Good Dinosaur
Name: title, dtype: object

In [ ]:
get_recommendations('Pulp Fiction').head(10)

In [ ]:
s_movies['cast'] = s_movies['cast'].apply(literal_eval)
s_movies['crew'] = s_movies['crew'].apply(literal_eval)
s_movies['keywords'] = s_movies['keywords'].apply(literal_eval)
s_movies['cast_size'] = s_movies['cast'].apply(lambda x: len(x))
s_movies['crew_size'] = s_movies['crew'].apply(lambda x: len(x))

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
s_movies['director'] = s_movies['crew'].apply(get_director)
s_movies['cast'] = s_movies['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
s_movies['cast'] = s_movies['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
s_movies['keywords'] = s_movies['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [ ]:
s_movies['cast'] = s_movies['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
s_movies['director'] = s_movies['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
s_movies['director'] = s_movies['director'].apply(lambda x: [x,x])


In [ ]:
s = s_movies.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-150-f19bdeb62a01>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = s_movies.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


murder        413
music         302
love          222
sex           219
friendship    202
Name: keyword, dtype: int64

In [ ]:
s = s[s > 1]
stemmer = SnowballStemmer('english')



In [ ]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words


In [ ]:
s_movies['keywords'] = s_movies['keywords'].apply(filter_keywords)
s_movies['keywords'] = s_movies['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
s_movies['keywords'] = s_movies['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [ ]:
s_movies['cast'].shape

(9219,)

In [ ]:
s_movies['director'].shape

(9219,)

In [ ]:
s_movies['genres'].shape

(9219,)

In [ ]:
s_movies['keywords'].shape

(9219,)

In [ ]:
s_movies['ls'] = pd.Series(list(s_movies['keywords']) + list(s_movies['cast']) + list(s_movies['director']) + list(s_movies['genres']))
s_movies['ls'] = s_movies['ls'].apply(lambda x: ' '.join(x))


In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(s_movies['ls'])


In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [ ]:
titles = s_movies['title']
indices = pd.Series(s_movies.index, index=s_movies['title'])

In [ ]:
get_recommendations('Toy Story').head(10)

192        Boys on the Side
7500    Hachi: A Dog's Tale
2817         The Odd Couple
25                  Othello
137                  Casper
1056               Heathers
1150                Tin Men
1342        One Night Stand
1721        St. Elmo's Fire
1797                   Toys
Name: title, dtype: object

In [ ]:
get_recommendations('Asoka').head(10)

634                                       Kim
8232                       Shakespeare Wallah
8919    The Second Best Exotic Marigold Hotel
5928                           Tai-Chi Master
4829                              Jungle Book
5481                                  Samsara
7693                            Eat Pray Love
8119           The Best Exotic Marigold Hotel
8357                     The World Before Her
1539                         The Last Emperor
Name: title, dtype: object

In [ ]:
get_recommendations('Interstellar').head(10)

8983                The Martian
987                       Alien
6252                   Serenity
7396                   Pandorum
7846                     BURN·E
8712    Guardians of the Galaxy
3076              Space Cowboys
3791                  SpaceCamp
3819             Mission to Mir
8934                       Home
Name: title, dtype: object

In [ ]:
get_recommendations('Pulp Fiction').head(10)

689          Bordello of Blood
2353            Risky Business
3605       Salome's Last Dance
5374    How to Steal a Million
5896             Frankenhooker
6937                  Flawless
133              Belle de Jour
616       The Great White Hype
1703                Snake Eyes
2571             The Hurricane
Name: title, dtype: object

In [ ]:
#collaborative based
reader = Reader()


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)


{'test_rmse': array([0.89675604, 0.89973857, 0.89683873, 0.89719102, 0.89501817]),
 'test_mae': array([0.69087196, 0.69409496, 0.69101347, 0.6898054 , 0.68848921]),
 'fit_time': (1.4300391674041748,
  1.4868276119232178,
  1.5047705173492432,
  1.429973840713501,
  1.3936774730682373),
 'test_time': (0.14892053604125977,
  0.19272685050964355,
  0.22525358200073242,
  0.16030454635620117,
  0.26390600204467773)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)


In [ ]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [ ]:
svd.predict(1, 500)


Prediction(uid=1, iid=500, r_ui=None, est=2.7957732958339556, details={'was_impossible': False})

In [ ]:
svd.predict(5, 7000)

Prediction(uid=5, iid=7000, r_ui=None, est=3.6100057074819354, details={'was_impossible': False})

In [ ]:
#Hybrid Model
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = pd.read_csv('links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(s_movies[['title', 'id']], on='id').set_index('title')

In [ ]:
indices_map = id_map.set_index('id')


In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:30]
    movie_indices = [i[0] for i in sim_scores]
    movies = s_movies.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)


In [ ]:
hybrid(5, 'Asoka')

,title,vote_count,vote_average,release_date,id,est
1658,The Little Mermaid,1921.0,7.2,1989-11-17,10144,4.335744
1163,Hamlet,118.0,7.3,1996-12-25,10549,4.277563
3519,The Lion in Winter,89.0,7.6,1968-01-01,18988,4.275379
232,Star Wars,6778.0,8.1,1977-05-25,11,4.209458
6901,Enchanted,1512.0,6.6,2007-11-20,4523,4.130914
521,Aladdin,3495.0,7.4,1992-11-25,812,4.080492
1597,Bambi,1450.0,6.8,1942-08-14,3170,3.964590
5184,The Prince & Me,286.0,5.9,2004-03-28,11137,3.953633
2991,Hamlet,95.0,6.7,1990-12-19,10264,3.951196
6844,I Could Never Be Your Woman,94.0,5.7,2007-05-11,15019,3.925324


In [ ]:
hybrid(700, 'Asoka')

,title,vote_count,vote_average,release_date,id,est
232,Star Wars,6778.0,8.1,1977-05-25,11,4.199728
1163,Hamlet,118.0,7.3,1996-12-25,10549,4.076926
3203,You Can Count on Me,89.0,6.8,2000-11-17,14295,4.066013
3519,The Lion in Winter,89.0,7.6,1968-01-01,18988,4.030395
6901,Enchanted,1512.0,6.6,2007-11-20,4523,3.828745
1658,The Little Mermaid,1921.0,7.2,1989-11-17,10144,3.754415
5184,The Prince & Me,286.0,5.9,2004-03-28,11137,3.738223
8760,The Look of Silence,75.0,8.0,2014-08-30,267480,3.703716
521,Aladdin,3495.0,7.4,1992-11-25,812,3.699274
8993,The Wolfpack,116.0,6.7,2015-06-12,307931,3.630656


In [ ]:
hybrid(2, 'Se7en')

,title,vote_count,vote_average,release_date,id,est
6759,21 Up,18.0,7.9,1977-01-01,20553,4.041990
3452,The Magnificent Seven,472.0,7.4,1960-10-22,966,3.977072
6686,7 Plus Seven,20.0,7.5,1970-01-01,50754,3.895472
6719,Zodiac,2080.0,7.3,2007-03-02,1949,3.833524
7183,Seven Pounds,2092.0,7.5,2008-12-18,11321,3.745727
3192,The Beyond,121.0,6.6,1981-04-22,19204,3.704547
1781,Cube,1101.0,6.9,1997-09-09,431,3.690517
6682,49 Up,12.0,6.9,2005-09-15,13365,3.685623
5079,Monster,500.0,7.0,2003-11-16,504,3.660844
4206,Blood Work,210.0,6.1,2002-08-04,9573,3.530212


In [ ]:
hybrid(1000, 'Se7en')

,title,vote_count,vote_average,release_date,id,est
1781,Cube,1101.0,6.9,1997-09-09,431,4.015843
6719,Zodiac,2080.0,7.3,2007-03-02,1949,3.963661
7183,Seven Pounds,2092.0,7.5,2008-12-18,11321,3.918167
6759,21 Up,18.0,7.9,1977-01-01,20553,3.874162
3452,The Magnificent Seven,472.0,7.4,1960-10-22,966,3.859401
6682,49 Up,12.0,6.9,2005-09-15,13365,3.801783
6686,7 Plus Seven,20.0,7.5,1970-01-01,50754,3.771910
3192,The Beyond,121.0,6.6,1981-04-22,19204,3.695064
5079,Monster,500.0,7.0,2003-11-16,504,3.690240
4307,The Ring,1720.0,6.5,2002-10-18,565,3.618327


In [ ]:
hybrid(45, 'Forrest Gump')

,title,vote_count,vote_average,release_date,id,est
4480,City of God,1852.0,8.2,2002-02-05,598,4.203082
2444,The General,240.0,8.0,1926-12-31,961,4.201885
6668,Letters from Iwo Jima,553.0,7.2,2006-12-19,1251,3.997594
1599,Dangerous Liaisons,267.0,7.1,1988-12-16,859,3.865155
2431,The Insider,489.0,7.3,1999-10-28,9008,3.807759
5788,The Music Man,47.0,6.7,1962-06-19,13671,3.714874
3338,Long Night's Journey Into Day,2.0,2.5,2000-01-29,95682,3.681092
4584,Salaam Bombay!,19.0,7.1,1988-08-24,45129,3.654959
6760,Maniac Cop,87.0,5.7,1988-05-13,14240,3.620852
2374,Niagara,84.0,6.7,1953-01-26,19997,3.610902


In [ ]:
hybrid(7745, 'Forrest Gump')

,title,vote_count,vote_average,release_date,id,est
4480,City of God,1852.0,8.2,2002-02-05,598,4.271636
2444,The General,240.0,8.0,1926-12-31,961,4.156230
6668,Letters from Iwo Jima,553.0,7.2,2006-12-19,1251,3.992251
1599,Dangerous Liaisons,267.0,7.1,1988-12-16,859,3.894225
8541,Frozen,5440.0,7.3,2013-11-27,109445,3.886350
2431,The Insider,489.0,7.3,1999-10-28,9008,3.885993
5788,The Music Man,47.0,6.7,1962-06-19,13671,3.717516
6862,Lars and the Real Girl,517.0,7.1,2007-10-12,6615,3.702826
4584,Salaam Bombay!,19.0,7.1,1988-08-24,45129,3.631173
3338,Long Night's Journey Into Day,2.0,2.5,2000-01-29,95682,3.627211


In [ ]:
hybrid(5, 'Salaam Bombay!')

,title,vote_count,vote_average,release_date,id,est
1347,Boogie Nights,812.0,7.5,1997-09-11,4995,4.359127
1055,The Big Sleep,244.0,7.6,1946-08-23,910,4.349213
4898,Wonderland,73.0,6.2,2003-10-23,4997,4.239191
695,Cyclo,23.0,6.8,1995-01-01,36266,4.182581
138,Clockers,69.0,6.8,1995-09-15,20649,4.111646
2637,Hard Boiled,169.0,7.4,1992-04-16,11782,4.087575
2086,Go,198.0,7.0,1999-04-09,9430,3.995462
1502,Whatever,2.0,8.3,1998-07-10,102304,3.897360
3078,Saving Grace,115.0,6.3,2000-01-24,2360,3.883424
1238,"Truth or Consequences, N.M.",15.0,5.7,1997-01-01,31017,3.848024


In [ ]:
hybrid(500, 'Salaam Bombay!')

,title,vote_count,vote_average,release_date,id,est
1055,The Big Sleep,244.0,7.6,1946-08-23,910,3.757296
4790,Valley of the Dolls,30.0,6.7,1967-12-27,3055,3.496203
3830,Life as a House,122.0,7.2,2001-10-25,11457,3.491901
2086,Go,198.0,7.0,1999-04-09,9430,3.484672
4898,Wonderland,73.0,6.2,2003-10-23,4997,3.370398
695,Cyclo,23.0,6.8,1995-01-01,36266,3.313022
3078,Saving Grace,115.0,6.3,2000-01-24,2360,3.219300
2637,Hard Boiled,169.0,7.4,1992-04-16,11782,3.183967
2985,Fatal Beauty,15.0,5.3,1987-10-30,35463,3.162445
405,Fresh,56.0,7.6,1994-08-24,13815,3.160219
